# Module 6

## Video 27: Multiple Vessels in a Movement
**Python for the Energy Industry**

Sometimes, a Cargo Movement will contain information on multiple vessels. There are two reasons this might happen:
- The cargo is loaded to, or unloaded from, a floating storage vessel
- A ship to ship transfer (STS) occurs

Let's find some example Cargo Movements where this is the case.

[Cargo Movements documentation](https://vortechsa.github.io/python-sdk/endpoints/cargo_movements/)

In [1]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import vortexasdk as v
now = datetime.utcnow()

We start as before with a simple query:

In [2]:
cm_query = v.CargoMovements().search(
    filter_activity="loading_state",
    filter_time_min=now,
    filter_time_max=now)

print(len(cm_query))

2020-12-10 19:14:42,421 vortexasdk.client — WARNING — You are using vortexasdk version 0.26.0, however version 0.27.1 is available.
You should consider upgrading via the 'pip install vortexasdk --upgrade' command.


Loading from API: 1000it [00:01, 770.21it/s]             

782


We can find Cargo Movements with STS events by getting those there multiple vessels are detailed:

In [3]:
multi_vessel = [q for q in cm_query if len(q['vessels']) > 1]
print(len(multi_vessel))

31


Let's take a look at the types of event happening in each of these:

In [4]:
for cm in multi_vessel:
    event_types = [e['event_type'] for e in cm['events']]
    print(event_types)

['cargo_port_load_event', 'cargo_port_unload_event', 'cargo_sts_event']
['cargo_port_load_event', 'cargo_port_unload_event', 'cargo_sts_event']
['cargo_port_load_event', 'cargo_fso_unload_event']
['cargo_port_load_event', 'cargo_port_unload_event', 'cargo_sts_event', 'cargo_storage_event']
['cargo_port_load_event', 'cargo_fso_unload_event']
['cargo_port_load_event', 'cargo_port_unload_event', 'cargo_sts_event']
['cargo_port_load_event', 'cargo_port_unload_event', 'cargo_sts_event']
['cargo_port_load_event', 'cargo_port_unload_event', 'cargo_sts_event']
['cargo_port_load_event', 'cargo_port_unload_event', 'cargo_sts_event', 'cargo_storage_event']
['cargo_port_load_event', 'cargo_sts_event']
['cargo_port_load_event', 'cargo_port_unload_event', 'cargo_sts_event']
['cargo_port_load_event', 'cargo_sts_event', 'cargo_storage_event']
['cargo_port_load_event', 'cargo_port_unload_event', 'cargo_sts_event']
['cargo_port_load_event', 'cargo_port_unload_event', 'cargo_sts_event']
['cargo_port_load

## STS Events

We'll look in more detail at those Cargo Movements with STS events.

In [5]:
sts_movements = [cm for cm in multi_vessel if 'cargo_sts_event' in [e['event_type'] for e in cm['events']]]
print(len(sts_movements))

24


Let's take a look at the vessels in one of them:

In [6]:
sts_movements[0]['vessels'][0]

{'id': 'accd8fb1c8273f0005615e8906d0377d1f411da72cd7299d1545a8635bdf267c',
 'mmsi': 248108000,
 'imo': 9773478,
 'name': 'MARFA',
 'dwt': 159513,
 'cubic_capacity': 167100,
 'vessel_class': 'suezmax',
 'corporate_entities': [{'id': '018fca8c06e4ab32a0250c1f888023903bf4debdf4b94c592c07483b50b821cd',
   'label': 'CARDIFF MARINE',
   'layer': 'commercial_owner',
   'probability': 1,
   'source': 'external'}],
 'start_timestamp': '2020-10-31T14:49:25+0000',
 'end_timestamp': '2020-11-13T18:17:23+0000',
 'fixture_fulfilled': False,
 'voyage_id': '3765abac1a9690ccf470af2460b464146580e7f303c71ebdcfac7e0d528b819b',
 'tags': [],
 'status': 'vessel_status_laden_known',
 'year': 2017,
 'scrubber': [{'tag': 'vessel_scrubber_tag',
   'scrubber': 'da4495540c2cee36',
   'planned': False}],
 'flag': [{'tag': 'vessel_flag_tag',
   'flag': 'MT',
   'flag_country': '80dd61da7ce1edccaa43d2d60207c482e397bdd7f7efe7ad2a222d35dde2bc8c'}]}

In [7]:
sts_movements[0]['vessels'][1]

{'id': '28d362261f980497d701e30991fdb9ed72b62335932666039b900ef16bef6ecc',
 'mmsi': 563553000,
 'imo': 9253064,
 'name': 'EAGLE TUCSON',
 'dwt': 107123,
 'cubic_capacity': 120330,
 'vessel_class': 'aframax',
 'corporate_entities': [{'id': 'b63335a112fa31c048322c1556cfb336ef6268d0a6d3fe48319e0b6f99c3e377',
   'label': 'AET',
   'layer': 'commercial_owner',
   'probability': 1,
   'source': 'external'}],
 'start_timestamp': '2020-11-12T22:54:05+0000',
 'end_timestamp': '2020-11-19T02:59:33+0000',
 'voyage_id': '35b62c9a624a5b802fdb8e8f62b289049095a080b3bc8cee1c4b35184bd5a3dc',
 'tags': [],
 'status': 'vessel_status_laden_known',
 'year': 2003,
 'scrubber': [],
 'flag': [{'tag': 'vessel_flag_tag',
   'flag': 'SG',
   'flag_country': '7fed43c640957555ddac588be64822538078409a0acdaf22126623203ef9954a'}]}

 And also the STS event:

In [8]:
sts_movements[0]['events'][2]

{'event_type': 'cargo_sts_event',
 'from_vessel_id': 'accd8fb1c8273f0005615e8906d0377d1f411da72cd7299d1545a8635bdf267c',
 'from_vessel_name': 'MARFA',
 'to_vessel_id': '28d362261f980497d701e30991fdb9ed72b62335932666039b900ef16bef6ecc',
 'to_vessel_name': 'EAGLE TUCSON',
 'location': [{'id': '2d92cc08f22524dba59f6a7e340f132a9da0ce9573cca968eb8e3752ef17a963',
   'layer': 'country',
   'label': 'United States',
   'source': 'model',
   'probability': 0.9840425},
  {'id': 'ae06d7e47d6ca2a5bcd7625b8ad402d39e5e92fcc740157dc3301a6d2a6bb207',
   'layer': 'region',
   'label': 'North America',
   'source': 'model',
   'probability': 0.9840425},
  {'id': '3f0a65d97728cdd1aaa5830d2de018d4378f2c453a89bc53c3c7009d8c02355a',
   'layer': 'shipping_region',
   'label': 'Gulf of Mexico',
   'source': 'model',
   'probability': 0.9840425},
  {'id': '4e79eb8e84d26f5c3c0006283bc1aa52c170b58d667be9848e136afea91a57e9',
   'layer': 'shipping_region',
   'label': 'PADD 3 (US Gulf Coast)',
   'source': 'model'

Let's view the data on movements with STS transfers in a DataFrame. The easiest way to do this is to convert our initial search into a DataFrame using the `to_df()` function, and then drop all movements with no STS event.

In [9]:
required_columns = ["vessels.0.name","vessels.0.vessel_class","vessels.1.name","vessels.1.vessel_class","product.group.label","quantity",
                    "events.cargo_port_load_event.0.location.port.label","events.cargo_sts_event.0.location.sts_zone.label",
                    "events.cargo_port_unload_event.0.location.port.label"]

new_labels = ["vessel0","vessel0_class","vessel1","vessel1_class","product_group","quantity","loading_port","sts_location","unloading_port"]

relabel = dict(zip(required_columns,new_labels))

In [10]:
cm_df = cm_query.to_df(columns=required_columns).rename(relabel,axis=1)

In [11]:
cm_df_multi = cm_df.dropna(subset=['loading_port','sts_location','unloading_port'])

In [12]:
cm_df_multi

,vessel0,vessel0_class,vessel1,vessel1_class,product_group,quantity,loading_port,sts_location,unloading_port
54,MARFA,suezmax,EAGLE TUCSON,aframax,Crude/Condensates,629149,Liza Field [GY],Southtex Lightering/STS Zone [US],"Houston, TX [US]"
96,DHONOUSSA,panamax,NS AFRICA,aframax,Dirty Petroleum Products,174409,Suez [EG],Malta STS [MT],"Pascagoula, MS [US]"
158,MARFA,suezmax,EAGLE TURIN,aframax,Crude/Condensates,472842,Liza Field [GY],Southtex Lightering/STS Zone [US],"Port Allen, LA [US]"
246,LOVINA,aframax,NISSOS RHENIA,vlcc_plus,Crude/Condensates,233144,Marsa Bashayer [SD],Fujairah STS [AE],Singapore [SG]
321,EAGLE BLANE,suezmax,APOLLONAS,vlcc_plus,Crude/Condensates,535941,Heidrun Field [NO],Scapa Flow STS [GB],Qingdao [CN]
331,SEAVIOLET,suezmax,OCEANA RIVER,aframax,Crude/Condensates,582806,Arzew [DZ],Linggi STS [MY],Singapore [SG]
448,RIO 2016,suezmax,CAP PHILIPPE,suezmax,Crude/Condensates,1013871,Sururu-Berbigao Field [BR],"Acu Petroleo T1/T-OIL (Prumo, Oiltanking)",Sines [PT]
482,VITIS,tiny_tanker,CHEMSTAR STELLAR,general_purpose,Clean Petroleum Products,24398,Palermo [IT],Suape STS [BR],Yenikoy-Autoport [TR]
572,LOVINA,aframax,NISSOS RHENIA,vlcc_plus,Crude/Condensates,436480,Marsa Bashayer [SD],Fujairah STS [AE],Singapore [SG]
619,FRONT SIRIUS,aframax,STI KINGSWAY,aframax,Crude/Condensates,575120,Oguendjo Terminal [GA],Sikka Anchorage STS,Botany Bay Nsw [AU]


### Exercise

Expand the range of the Cargo Movements - say to a ~2 week period. Can you find any movements involving 3 or more vessels? If so, isolate and have a look at these.